### 使用最小二乘法拟和曲线

高斯于1823年在误差$e_1,…,e_n$独立同分布的假定下,证明了最小二乘方法的一个最优性质: 在所有无偏的线性估计类中,最小二乘方法是其中方差最小的！
对于数据$(x_i, y_i)   (i=1, 2, 3...,m)$
拟合出函数$h(x)$
有误差，即残差：$r_i=h(x_i)-y_i$

此时$L2$范数(残差平方和)最小时，$h(x)$ 和 $y$ 相似度最高，更拟合

一般的$H(x)$为$n$次的多项式，$H(x)=w_0+w_1x+w_2x^2+...w_nx^n$

$w(w_0,w_1,w_2,...,w_n)$为参数

最小二乘法就是要找到一组 $w(w_0,w_1,w_2,...,w_n)$ ，使得$\sum_{i=1}^n(h(x_i)-y_i)^2$ (残差平方和) 最小

即，求 $min\sum_{i=1}^n(h(x_i)-y_i)^2$

----

举例：我们用目标函数$y=sin2{\pi}x$, 加上一个正态分布的噪音干扰，用多项式去拟合

In [ ]:
import numpy as np
import scipy as sp
from scipy.optimize import leastsq
import matplotlib.pyplot as plt
%matplotlib inline

* ps: numpy.poly1d([1,2,3])  生成  $1x^2+2x^1+3x^0$*

In [ ]:
# 目标函数
def real_func(x):
    return np.sin(2*np.pi*x)

# 多项式
def fit_func(p, x):
    f = np.poly1d(p)
    return f(x)

# 残差
def residuals_func(p, x, y):
    ret = fit_func(p, x) - y
    return ret

In [ ]:
# 十个点
np.random.seed(1) # set the random seed to make the running result reproducible.
x = np.linspace(0, 1, 10)
x_points = np.linspace(0, 1, 1000)
# 加上正态分布噪音的目标函数的值
y_ = real_func(x)
y = [np.random.normal(0, 0.1) + y1 for y1 in y_]

# scipy.optimize.leastsq() Usage

scipy.optimize.leastsq(func, x0, args=(), Dfun=None, full_output=0, col_deriv=0, ftol=1.49012e-08, xtol=1.49012e-08, gtol=0.0, maxfev=0, epsfcn=None, factor=100, diag=None)[source]  
Minimize the sum of squares of a set of equations.  
如上所述，leastsq()用于优化（最小化）一个方程组的平方和。  
它的第一个参数为待优化的函数，在本例中就是误差函数--residual = f(X) - y.  
x0表示该函数的参数的初始值，在本例中就是多项式的系数，函数func代表的是多项式，它决定于多项式系数。优化的目标就是要找到最优的多项式系数使得func(X,y)的平方和最小.  
args=()则是func的额外的参数,在本例中就是输入数据样本及其对应的参考输出。  

所以要利用scipy的leastsq来求解least-square fitting/estimation/optimization问题，首先要定义代表残留误差的函数。


In [ ]:
def fitting(M=0):
    """
    M    为 多项式的次数
    """
    # 随机初始化多项式参数
    p_init = np.random.rand(M + 1)
    # 最小二乘法
    p_lsq = leastsq(residuals_func, p_init, args=(x, y))
    print('Fitting Parameters:', p_lsq[0])

    # 可视化
    plt.plot(x_points, real_func(x_points), label='real')
    plt.plot(x_points, fit_func(p_lsq[0], x_points), label='fitted curve')
    plt.plot(x, y, 'bo', label='noise')
    plt.legend()
    return p_lsq

### M=0

In [ ]:
# M=0
p_lsq_0 = fitting(M=0)

### M=1

In [ ]:
# M=1
p_lsq_1 = fitting(M=1)

### M=3 

In [ ]:
# M=3
p_lsq_3 = fitting(M=3)

### M=9

In [ ]:
# M=9
p_lsq_9 = fitting(M=9)

当M=9时，多项式曲线通过了每个数据点，但是造成了过拟合

### 正则化

结果显示过拟合， 引入正则化项(regularizer)，降低过拟合

$Q(x)=\sum_{i=1}^n(h(x_i)-y_i)^2+\lambda||w||^2$。

回归问题中，损失函数是平方损失，正则化可以是参数向量的L2范数,也可以是L1范数。

- L1: regularization\*abs(p)

- L2: 0.5 \* regularization \* np.square(p)

In [ ]:
regularization = 0.0001


def residuals_func_regularization(p, x, y):
    ret = fit_func(p, x) - y
    ret = np.append(ret,
                    np.sqrt(0.5 * regularization * np.square(p)))  # L2范数作为正则化项
    return ret

In [ ]:
# 最小二乘法,加正则化项
p_init = np.random.rand(9 + 1)
p_lsq_regularization = leastsq(
    residuals_func_regularization, p_init, args=(x, y))

In [ ]:
plt.plot(x_points, real_func(x_points), label='real')
plt.plot(x_points, fit_func(p_lsq_9[0], x_points), label='fitted curve')
plt.plot(
    x_points,
    fit_func(p_lsq_regularization[0], x_points),
    label='regularization')
plt.plot(x, y, 'bo', label='noise')
plt.legend()

# Naive normal equation solution  

y = Xw  
w = {(X.T @ X)**(-1)}@(X.T @ y)

## 1. A first toy program with M=3 

In [ ]:
## Design Matrix
print(x)
M = 3
X = np.zeros((len(x),M+1))
X[:,0] = 1
X[:,1] = x
X[:,2] = x**2
X[:,3] = x**3
print(X)
w_ls = np.linalg.inv(X.T@X) @ (X.T @ y)
print(w_ls)
print(p_lsq_3)

对比w_ls与前面得到的p_lsq_3可以知道他们在误差范围以内是相等的，但是系数排列顺序不相同。系数排顺序是以数据的排列顺序相对应的。  
由于在design_matrix(X)中，第1列置为全1（对应intercept term），而第2列为一次项。。。，因此最后求得的系数也是这样的对应关系。

## 2. Function packaging with 'M' as hyper-parameter 

In [ ]:
def direct_ls(x,y,M=3):
    X = np.zeros((len(x),M+1))
    for k in range(M+1):
        X[:,k] = x**k

    w_ls = np.linalg.inv(X.T@X) @ (X.T @ y)
    
    return w_ls

In [ ]:
print('M = ',M,': ',direct_ls(x,y,3))
print('M = ',M,': ',direct_ls(x,y,6))
print('M = ',M,': ',direct_ls(x,y,9))